In [1]:
import pandas as pd
import sqlalchemy.orm as so
import sqlalchemy as sa

from dataclasses import asdict
from dotenv import load_dotenv
from orm_loader.helpers import configure_logging
from omop_alchemy import get_engine_name, TEST_PATH, ROOT_PATH
from omop_alchemy.cdm.model.vocabulary import Concept, Concept_Ancestor, Concept_Relationship, Concept_Synonym

from omop_graph.graph.kg import KnowledgeGraph
from omop_graph.render import (
    render_subgraph,
    render_trace,
    render_path,
    render_explained_path,
    bind_default_renderers,
)
from omop_graph.reasoning.phenotypes import (
    find_common_parents,
)

At the moment, this demo is input to a manual process of hierarchy resolution, because there are certainly arguments for allowing impurity (e.g. SNOMED cancer conditions do not meet the original ICDO3-specific filter if there is no corresponding ICDO3 code), as well as for supporting less than full coverage (e.g. valid but unusual/atypical classifications that introduce a lot of noise) - TBD the ideal level of final automation here, but irrespectively it is is still relatively useful for initial orientation steps...

In [2]:
configure_logging()
load_dotenv()

engine_string = get_engine_name()
engine = sa.create_engine(engine_string, future=True, echo=False)

2026-01-12 10:18:45,944 | INFO     | sql_loader.omop_alchemy.config | Database engine configured


In [3]:
Session = so.sessionmaker(bind=engine)

session = Session()
kg = KnowledgeGraph(session)
bind_default_renderers(kg)

In [4]:
seeds = [4173456, 4174251, 4174241, 4172043, 4173614, 4175026] # ecog 0 thru 5
parent_stats = find_common_parents(seeds=seeds, kg=kg)
candidate_df = pd.DataFrame([{'concept_id': c} | asdict(v) for c, v in parent_stats.items()])

In [5]:
candidate_df

,concept_id,descendants,found,coverage,pollution,completeness,purity,max_depth
0,4308794,"{4174241, 4172043, 4174251, 4173614, 4173456, ...","{4174241, 4172043, 4174251, 4173614, 4173456, ...",6,0,1.0,1.000000,1
1,432453,"{3655685, 4294662, 4032527, 4032528, 4032529, ...","{4174241, 4172043, 4174251, 4173614, 4173456, ...",6,986,1.0,0.006048,2
2,4089214,"{46284803, 4153350, 4120591, 3661842, 3661843,...","{4174241, 4172043, 4174251, 4173614, 4173456, ...",6,4195,1.0,0.001428,5
3,4041284,"{4259853, 4161577, 4161578, 4161579, 4128815, ...","{4174241, 4172043, 4174251, 4173614, 4173456, ...",6,7135,1.0,0.000840,5
4,4094294,"{4259853, 4161577, 4161578, 4161579, 4227116, ...","{4174241, 4172043, 4174251, 4173614, 4173456, ...",6,14450,1.0,0.000415,4


In [6]:
seeds = [4230556, 4085075, 434489, 4173456, 4174251, 4174241, 4172043, 4173614, 4175026] # alive, dead, at risk
parent_stats = find_common_parents(seeds=seeds, kg=kg)
candidate_df = pd.DataFrame([{'concept_id': c} | asdict(v) for c, v in parent_stats.items()])

In [7]:
candidate_df

,concept_id,descendants,found,coverage,pollution,completeness,purity,max_depth
0,432453,"{3655685, 4294662, 46270485, 35610650, 4202524...","{4174241, 4172043, 4174251, 4173614, 4173456, ...",9,538,1.000000,0.016453,2
1,4308794,"{4174241, 4172043, 4174251, 4173614, 4173456, ...","{4174241, 4172043, 4174251, 4173614, 4173456, ...",6,0,0.666667,1.000000,1
2,4089214,"{46284803, 4153350, 4120591, 3661842, 3661843,...","{4174241, 4172043, 4174251, 4173614, 4173456, ...",6,4195,0.666667,0.001428,5
3,4041284,"{4259853, 4161577, 4161578, 4161579, 4128815, ...","{4174241, 4172043, 4174251, 4173614, 4173456, ...",6,7135,0.666667,0.000840,5
4,4094294,"{4259853, 4161577, 4161578, 4161579, 4227116, ...","{4174241, 4172043, 4174251, 4173614, 4173456, ...",6,14450,0.666667,0.000415,4


In [8]:
head_neck_conditions = [Concept.concept_code.ilike(f'%C{k:02d}%') for k in range(0, 15)]
head_neck_topog = [Concept.concept_code==f'C{k:02d}' for k in range(0, 15)]

In [9]:
topog_concepts = pd.DataFrame(
    session.query(
        *Concept.__table__.columns
    )
    .filter(Concept.vocabulary_id=='ICDO3')
    .filter(sa.or_(*head_neck_topog))
)

In [10]:
hn_concepts = pd.DataFrame(
    session.query(
        *Concept.__table__.columns
    )
    .filter(Concept.vocabulary_id=='ICDO3')
    .filter(Concept.concept_class_id=='ICDO Condition')
    .filter(sa.or_(*head_neck_conditions))
    .filter(Concept.concept_code.like('%/3-%'))
)

In [11]:
all_parents = {}
all_candidates = {}

for topography in topog_concepts.itertuples():
    code = topography.concept_code
    seeds = set(hn_concepts[hn_concepts.concept_code.str.contains(code)&hn_concepts.invalid_reason.isna()].concept_id)
    print(f"{code}: {topography.concept_name} ({len(seeds)} candidates)")
    all_parents[code] = find_common_parents(seeds=seeds, kg=kg)
    all_candidates[code] = pd.DataFrame([{'concept_id': c} | asdict(v) for c, v in all_parents[code].items()])
    best_result = all_candidates[code].sort_values(['coverage', 'purity']).tail(5)
    display(best_result)

C00: Lip (734 candidates)


,concept_id,descendants,found,coverage,pollution,completeness,purity,max_depth
159,135858,"{36528135, 36519943, 36536330, 4145173, 365609...","{36562945, 36532227, 36546565, 36517893, 36519...",718,775,0.978202,0.480911,7
136,4116808,"{36528135, 36519943, 36536330, 36560921, 41779...","{36562945, 36532227, 36546565, 36517893, 36519...",718,694,0.978202,0.508499,6
135,37161350,"{36519943, 36528135, 36536330, 36560921, 36544...","{36562945, 36532227, 36546565, 36517893, 36519...",718,520,0.978202,0.579968,6
101,4054605,"{602113, 4313090, 602115, 36532227, 602114, 36...","{36562945, 36532227, 36546565, 36517893, 36519...",718,509,0.978202,0.585167,5
54,4180779,"{602113, 4313090, 602115, 36532227, 602114, 36...","{36562945, 36532227, 36546565, 36517893, 36519...",718,240,0.978202,0.749478,4


C01: Base of tongue (144 candidates)


,concept_id,descendants,found,coverage,pollution,completeness,purity,max_depth
69,4178962,"{36560896, 36528130, 36552707, 36524036, 36536...","{36566017, 36562435, 36561419, 36555283, 36536...",142,1033,0.986111,0.120851,4
33,437498,"{36560896, 36528130, 36552707, 36536324, 36524...","{36566017, 36562435, 36561419, 36555283, 36536...",142,985,0.986111,0.125998,3
87,4054607,"{36566017, 4248065, 36562435, 4158473, 3656141...","{36566017, 36562435, 36561419, 36555283, 36536...",142,40,0.986111,0.780220,4
34,4181332,"{36566017, 4248065, 36562435, 4158473, 3656141...","{36566017, 36562435, 36561419, 36555283, 36536...",142,13,0.986111,0.916129,3
0,256633,"{36566017, 36562435, 36561419, 36555283, 36536...","{36566017, 36562435, 36561419, 36555283, 36536...",142,4,0.986111,0.972603,2


C02: Other and unspecified parts of tongue (972 candidates)


...

,concept_id,descendants,found,coverage,pollution,completeness,purity,max_depth
46,4181342,"{44498945, 36534274, 36558851, 36517896, 36556...","{44498945, 36567557, 36567567, 36539926, 36534...",133,710,0.992537,0.157770,5
31,439746,"{44498945, 36534274, 36558851, 36517896, 36556...","{44498945, 36567557, 36567567, 36539926, 36534...",133,676,0.992537,0.164400,4
38,4129252,"{44498945, 36560901, 36567557, 36567567, 43156...","{44498945, 36567557, 36567567, 36539926, 36534...",133,31,0.992537,0.810976,4
32,4180789,"{44498945, 36567557, 36567567, 4315668, 365399...","{44498945, 36567557, 36567567, 36539926, 36534...",133,5,0.992537,0.963768,4
1,435190,"{44498945, 36567557, 36567567, 36539926, 36534...","{44498945, 36567557, 36567567, 36539926, 36534...",133,1,0.992537,0.992537,2


C13: Hypopharynx (669 candidates)


,concept_id,descendants,found,coverage,pollution,completeness,purity,max_depth
249,4129247,"{44498945, 36534274, 36558851, 36560901, 36517...","{36534274, 36558851, 36517896, 36556809, 36548...",651,378,0.973094,0.632653,8
212,44783159,"{44498945, 36534274, 36558851, 36560901, 36517...","{36534274, 36558851, 36517896, 36556809, 36548...",651,366,0.973094,0.640118,8
154,4129250,"{44498945, 36534274, 36558851, 36560901, 36517...","{36534274, 36558851, 36517896, 36556809, 36548...",651,359,0.973094,0.644554,6
99,4181342,"{44498945, 36534274, 36558851, 36517896, 36556...","{36534274, 36558851, 36517896, 36556809, 36548...",651,192,0.973094,0.772242,5
4,439746,"{44498945, 36534274, 36558851, 36517896, 36556...","{36534274, 36558851, 36517896, 36556809, 36548...",651,158,0.973094,0.804697,4


C14: Other and ill-defined sites in lip, oral cavity and pharynx (384 candidates)


,concept_id,descendants,found,coverage,pollution,completeness,purity,max_depth
132,4115106,"{42598582, 42598955, 42599092, 42599192, 43266...","{36564997, 36526091, 36528139, 36556815, 36522...",379,22528,0.986979,0.016545,7
17,4111017,"{4194462, 42598582, 4194827, 132258, 42599689,...","{36564997, 36526091, 36528139, 36556815, 36522...",379,22419,0.986979,0.016624,7
92,4028253,"{42598582, 42599844, 132565, 132573, 42599972,...","{36564997, 36526091, 36528139, 36556815, 36522...",379,22101,0.986979,0.016859,6
18,196931,"{44498945, 36536321, 44498948, 36536324, 44498...","{36564997, 36526091, 36528139, 36556815, 36522...",379,18788,0.986979,0.019774,5
128,4184252,"{44498945, 4096016, 4194323, 36536341, 4449896...","{36564997, 36526091, 36528139, 36556815, 36522...",379,6049,0.986979,0.058961,9
